In [10]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np 

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
s3 = boto3.resource('s3')
bucket_name = "rachaeld-data445"
bucket = s3.Bucket(bucket_name)

file_key = 'College.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the data-file
college = pd.read_csv(file_content_stream)
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [11]:
#1b
college['Private'].replace(['Yes', 'No'],
                        [1,0], inplace=True)
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,1,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,1,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,1,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [12]:
#1c
# defining the input and target variables
X = college[['Private', 'F.Undergrad', 'P.Undergrad', 'Outstate', 'Room.Board', 'Books', 'Personal', 'S.F.Ratio', 'Grad.Rate']]
Y = college['Apps']

# splitting the data
X_train, X_test, Y_train,Y_test = train_test_split(X,Y,test_size = .2)

In [13]:
#1d
# min-max transformation
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [16]:
#1e
# building the linear model 
lm_md = LinearRegression().fit(X_train, Y_train)

# predicting on test
lm_pred = lm_md.predict(X_test)

#computing the mse of the linear regression model
mse1 = np.mean(np.power(Y_test-lm_pred, 2))
print('The mse of the linear model is:', mse1)

The mse of the linear model is: 4542941.031835315


In [22]:
#1f
# Ridge regression
ridge_cv = RidgeCV(alphas = np.linspace(0.001, 100, num = 100), cv= 5).fit(X_train, Y_train)

# extracting the best lambda
cv_lambda = ridge_cv.alpha_
print('The best lambda of the ridge model is:', cv_lambda)

# building the ridge model
ridge_md = Ridge(alpha = cv_lambda).fit(X_train, Y_train)

# predicting on test
ridge_pred = ridge_md.predict(X_test)

# computing the mse
mse2 = np.mean(np.power(Y_test-ridge_pred, 2))
print('The mse of the ridge model is:', mse2)

The best lambda of the ridge model is: 1.011090909090909
The mse of the ridge model is: 4177046.165680119


In [23]:
#1g
# estimating lanbda for lasso
lasso_cv = LassoCV(alphas = np.linspace(0.001, 100, num = 100), normalize = True, cv= 5).fit(X_train, Y_train)

# extracting the best lambda
cv_lambda = lasso_cv.alpha_
print('The estimated lambda for the lasso model is:', cv_lambda)
    
# building lasso model
lasso_md = Lasso(alpha= cv_lambda, normalize = True).fit(X_train,Y_train)
lasso_md.coef_

# predicting on test
lasso_pred = lasso_md.predict(X_test)

# computing the mse
mse3 = np.mean(np.power(Y_test-lasso_pred, 2))
print('The mse of the lasso model is:', mse3)

The estimated lambda for the lasso model is: 5.051454545454546
The mse of the lasso model is: 4379160.549499563


/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

In [ ]:
#1h 
#model 1 linear regression mse:4542941.031835315
#model 2 ridge regression mse: 4177046.165680119
#model 3 LASSO regression mse: 4379160.549499563
# We would want to use model 2, the ridge regression model, because it has the smallest MSE value.